# Загрузка данных

https://www.kaggle.com/shubh0799/churn-modelling

https://drive.google.com/file/d/1Ym8EaWePP17RvrweKhWzCJmITXLgqvCZ/view?usp=sharing

In [ ]:
! gdown --id 1Ym8EaWePP17RvrweKhWzCJmITXLgqvCZ

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:138: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1Ym8EaWePP17RvrweKhWzCJmITXLgqvCZ
To: /content/churn-modeling.zip
100% 268k/268k [00:00<00:00, 105MB/s]


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/churn-modeling.zip')

# Обучающая, валидационная и тестовая выборка

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train, test = train_test_split(df,train_size=0.6,random_state=42,stratify=df['Exited'])

In [ ]:
val, test = train_test_split(test,train_size=0.5,random_state=42,stratify=test['Exited'])

# Обучение модели

In [ ]:
! pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 6.1 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostClassifier

In [ ]:
X = ['CustomerId', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary']

cat_features = ['Geography','Gender']

y = ['Exited']

In [ ]:
from catboost import Pool

train_data = Pool(data=train[X],
                  label=train[y],
                  cat_features=cat_features
                 )

valid_data = Pool(data=val[X],
                  label=val[y],
                  cat_features=cat_features
                 )

test_data = Pool(data=test[X],
                  label=test[y],
                  cat_features=cat_features
                 )

In [ ]:
params = {'verbose':100,
          'eval_metric':'AUC',
          'loss_function': 'Logloss',
          'random_seed':42,
          'learning_rate':0.01}

In [ ]:
model = CatBoostClassifier(**params)

In [ ]:
model.fit(train_data,eval_set=valid_data)

0:	test: 0.8338545	best: 0.8338545 (0)	total: 69ms	remaining: 1m 8s
100:	test: 0.8725090	best: 0.8725090 (100)	total: 1.12s	remaining: 9.98s
200:	test: 0.8784656	best: 0.8784656 (200)	total: 2.15s	remaining: 8.53s
300:	test: 0.8812676	best: 0.8812676 (300)	total: 2.98s	remaining: 6.92s
400:	test: 0.8823868	best: 0.8824376 (390)	total: 3.95s	remaining: 5.91s
500:	test: 0.8834491	best: 0.8834522 (498)	total: 6.26s	remaining: 6.24s
600:	test: 0.8834737	best: 0.8836554 (564)	total: 8.38s	remaining: 5.56s
700:	test: 0.8837847	best: 0.8838694 (685)	total: 10.3s	remaining: 4.41s
800:	test: 0.8834383	best: 0.8838694 (685)	total: 12.1s	remaining: 3.01s
900:	test: 0.8829318	best: 0.8838694 (685)	total: 14.1s	remaining: 1.55s
999:	test: 0.8822744	best: 0.8838694 (685)	total: 15.8s	remaining: 0us

bestTest = 0.8838694083
bestIteration = 685

Shrink model to first 686 iterations.


In [ ]:
n_iters = model.best_iteration_ + 1

In [ ]:
n_iters

686

In [ ]:
params = {'iterations':n_iters,
          'verbose':100,
          'eval_metric':'AUC',
          'loss_function': 'Logloss',
          'random_seed':42,
          'learning_rate':0.01}

In [ ]:
model = CatBoostClassifier(**params)

In [ ]:
train_full = pd.concat([train,val])

In [ ]:
train_full_data = Pool(train_full[X],
                       label=train_full[y],
                       cat_features=cat_features)

In [ ]:
model.fit(train_full_data)

0:	total: 27.6ms	remaining: 18.9s
100:	total: 1.42s	remaining: 8.21s
200:	total: 3.06s	remaining: 7.38s
300:	total: 4.97s	remaining: 6.36s
400:	total: 5.91s	remaining: 4.2s
500:	total: 6.85s	remaining: 2.53s
600:	total: 7.8s	remaining: 1.1s
685:	total: 9.25s	remaining: 0us


In [ ]:
test['y_score_no_cross_val'] = model.predict_proba(test_data)[:,1]

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(test['Exited'],test['y_score_no_cross_val'])

0.8735839074822127

# Кросс-валидация

https://github.com/catboost/tutorials/blob/master/classification/classification_tutorial.ipynb

In [ ]:
from catboost import cv

In [ ]:
params = {'verbose':100,
          'eval_metric':'AUC',
          'loss_function': 'Logloss',
          'random_seed':42,
          'learning_rate':0.01}

In [ ]:
cv_data = cv(
    params = params,
    pool = train_full_data,
    fold_count=5,
    shuffle=True,
    partition_random_seed=0,
    stratified=False,
    verbose=False
)

Training on fold [0/5]

bestTest = 0.866360207
bestIteration = 881

Training on fold [1/5]

bestTest = 0.8716621864
bestIteration = 998

Training on fold [2/5]

bestTest = 0.8695412245
bestIteration = 540

Training on fold [3/5]

bestTest = 0.8795945701
bestIteration = 942

Training on fold [4/5]

bestTest = 0.8586794872
bestIteration = 750



In [ ]:
cv_data

,iterations,test-AUC-mean,test-AUC-std,test-Logloss-mean,test-Logloss-std,train-Logloss-mean,train-Logloss-std
0,0,0.818479,0.009480,0.685020,0.000209,0.684939,0.000262
1,1,0.823848,0.010334,0.677216,0.000374,0.677055,0.000503
2,2,0.831819,0.009265,0.668989,0.000472,0.668799,0.000751
3,3,0.834559,0.008771,0.662008,0.001326,0.661753,0.000689
4,4,0.834483,0.008218,0.654599,0.001429,0.654270,0.000898
...,...,...,...,...,...,...,...
995,995,0.868703,0.007809,0.329986,0.016873,0.284097,0.003644
996,996,0.868715,0.007781,0.329972,0.016875,0.284057,0.003631
997,997,0.868718,0.007804,0.329972,0.016876,0.284003,0.003636
998,998,0.868725,0.007777,0.329969,0.016855,0.283943,0.003613


In [ ]:
cv_data[cv_data['test-AUC-mean'] == cv_data['test-AUC-mean'].max()]

,iterations,test-AUC-mean,test-AUC-std,test-Logloss-mean,test-Logloss-std,train-Logloss-mean,train-Logloss-std
741,741,0.868822,0.007457,0.330818,0.017005,0.297428,0.004326


In [ ]:
n_iters = cv_data[cv_data['test-AUC-mean'] == cv_data['test-AUC-mean'].max()]['iterations'].values[0]

In [ ]:
n_iters

741

In [ ]:
params = {'iterations':n_iters,
          'verbose':100,
          'eval_metric':'AUC',
          'loss_function': 'Logloss',
          'random_seed':42,
          'learning_rate':0.01}

In [ ]:
model = CatBoostClassifier(**params)

In [ ]:
model.fit(train_full_data)

0:	total: 10.3ms	remaining: 7.63s
100:	total: 1.15s	remaining: 7.28s
200:	total: 2.98s	remaining: 8.01s
300:	total: 4.13s	remaining: 6.04s
400:	total: 5.1s	remaining: 4.32s
500:	total: 6.06s	remaining: 2.9s
600:	total: 6.99s	remaining: 1.63s
700:	total: 7.93s	remaining: 452ms
740:	total: 8.3s	remaining: 0us


In [ ]:
test['y_score_cross_val'] = model.predict_proba(test_data)[:,1]

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(test['Exited'],test['y_score_cross_val'])

0.8737196364315009

In [ ]:
len(train_full)

8000

# Подбор гиперпараметров

https://github.com/catboost/catboost/blob/master/catboost/tutorials/events/2020_06_04_catboost_tutorial/catboost_features.ipynb

https://youtu.be/ZaP5qFSIcIw?t=2043

https://github.com/catboost/catboost/blob/master/catboost/tutorials/hyperparameters_tuning/hyperparameters_tuning.ipynb

In [ ]:
model.get_all_params()

{'nan_mode': 'Min',
 'eval_metric': 'AUC',
 'combinations_ctr': ['Borders:CtrBorderCount=15:CtrBorderType=Uniform:TargetBorderCount=1:TargetBorderType=MinEntropy:Prior=0/1:Prior=0.5/1:Prior=1/1',
  'Counter:CtrBorderCount=15:CtrBorderType=Uniform:Prior=0/1'],
 'iterations': 1000,
 'sampling_frequency': 'PerTree',
 'fold_permutation_block': 0,
 'leaf_estimation_method': 'Newton',
 'random_score_type': 'NormalWithModelSizeDecrease',
 'counter_calc_method': 'SkipTest',
 'grow_policy': 'SymmetricTree',
 'penalties_coefficient': 1,
 'boosting_type': 'Plain',
 'model_shrink_mode': 'Constant',
 'feature_border_type': 'GreedyLogSum',
 'ctr_leaf_count_limit': 18446744073709551615,
 'bayesian_matrix_reg': 0.10000000149011612,
 'one_hot_max_size': 2,
 'eval_fraction': 0,
 'force_unit_auto_pair_weights': False,
 'l2_leaf_reg': 3,
 'random_strength': 1,
 'rsm': 1,
 'boost_from_average': False,
 'max_ctr_complexity': 4,
 'model_size_reg': 0.5,
 'simple_ctr': ['Borders:CtrBorderCount=15:CtrBorderType

In [ ]:
params = {'verbose':100,
          'eval_metric':'AUC',
          'loss_function': 'Logloss',
          'random_seed':42}

In [ ]:
model = CatBoostClassifier(**params)

In [ ]:
grid = {'learning_rate': [0.01, 0.1],
        'depth': [5, 6]}

In [ ]:
train_full_data

In [ ]:
result = model.grid_search(grid, train_full_data, verbose=False)

0:	test: 0.7811641	best: 0.7811641 (0)	total: 17.2ms	remaining: 17.2s
100:	test: 0.8435462	best: 0.8435769 (99)	total: 1.23s	remaining: 11s
200:	test: 0.8508923	best: 0.8508923 (200)	total: 2.57s	remaining: 10.2s
300:	test: 0.8547897	best: 0.8547897 (300)	total: 3.61s	remaining: 8.38s
400:	test: 0.8569385	best: 0.8569385 (400)	total: 4.42s	remaining: 6.61s
500:	test: 0.8576410	best: 0.8576410 (500)	total: 5.25s	remaining: 5.23s
600:	test: 0.8585538	best: 0.8585538 (600)	total: 6.11s	remaining: 4.05s
700:	test: 0.8590923	best: 0.8591487 (694)	total: 6.93s	remaining: 2.96s
800:	test: 0.8593590	best: 0.8595564 (749)	total: 7.76s	remaining: 1.93s
900:	test: 0.8593282	best: 0.8595564 (749)	total: 8.57s	remaining: 942ms
999:	test: 0.8593897	best: 0.8595974 (986)	total: 9.39s	remaining: 0us

bestTest = 0.8595974359
bestIteration = 986

Metric AUC is not calculated on train by default. To calculate this metric on train, add hints=skip_train~false to metric parameters.
0:	test: 0.7811641	best: 

In [ ]:
result['params']

{'depth': 5, 'learning_rate': 0.1}

In [ ]:
pd.DataFrame(result['cv_results'])['test-AUC-mean'].max()

0.8689660773549619

# Принципы sklearn

https://towardsdatascience.com/how-to-use-sklearn-pipelines-for-ridiculously-neat-code-a61ab66ca90d

# Преодбработка и фичеинжениринг

https://catboost.ai/en/docs/concepts/quantization

https://www.kaggle.com/learn/feature-engineering

# Калибровка

https://www.kaggle.com/residentmario/notes-on-classification-probability-calibration

# Catboost, тексты и эмбеддинги

https://github.com/catboost/catboost/blob/master/catboost/tutorials/events/2020_06_04_catboost_tutorial/text_features.ipynb

https://youtu.be/ZaP5qFSIcIw?t=3802

# Главные навыки для аналитика

- SQL
- pandas
- Основы статистики (AB тесты)

# Главные навыки для Дата Саентиста

- Все то же, что и у аналитика
- Машинное обучение (sklearn, catboost)
- более продвинутый python (основы ООП)
- linux, git, docker, веб-сервисы

# Как развиваться дальше?

- [Прикладное машинное обучение с помощью Scikit-Learn, Keras и TensorFlow: концепции, инструменты и техники для создания интеллектуальных систем, 2-е издание | Жерон Орельен](https://www.ozon.ru/product/prikladnoe-mashinnoe-obuchenie-s-pomoshchyu-scikit-learn-keras-i-tensorflow-kontseptsii-207392052/?sh=HC92ZQyhUQ)
- Задачи на kaggle, вот [список](https://www.kaggle.com/getting-started/114864) для начинающих
- Откликайся на вакансии и решай тестовые. Можешь смотреть [тестовые по дс у меня на канале](https://www.youtube.com/playlist?list=PLQJ7ptkRY-xbefSg1XN3FA-SdSRFcCQfn)
- [Мой курс по статистике](https://www.youtube.com/playlist?list=PLQJ7ptkRY-xbHLLI66KdscKp_FJt0FsIi)
- Задачи по pandas и sql на [StrataScratch](https://www.stratascratch.com/?via=gleb)
- [Мой курс по SQL](https://www.udemy.com/course/sql-with-gleb/?referralCode=C99EC81AE75FEC9F50A2)
- Задачи на [itresume](https://itresume.ru/)
- Задачи на [Checkio](https://checkio.org/)
- Задачи на [Leetcode](https://leetcode.com/) вот по этому [списку](https://seanprashad.com/leetcode-patterns/)
- Мой [плейлист](https://www.youtube.com/playlist?list=PLQJ7ptkRY-xZ4qiXlmQQLgAPyXJhQ7HxZ) по основам Linux
- [Основы Git](https://youtu.be/0cGIiA0AjNw)
- [Основы докера](https://www.youtube.com/playlist?list=PLQJ7ptkRY-xbR0ka2TUxJkXna40XWu92m)
- [Основы веб-сервисов](https://www.youtube.com/playlist?list=PLQJ7ptkRY-xYLEAC5Y_sKqrJ9RA-U7Dja)

# Дополнительные материалы

- [Как работает градиентный бустинг](https://youtu.be/ZNJ3lKyI-EY)
- [Как учить английский](https://youtu.be/f9Q4Mwvd5pc)
- [Как искать работу](https://deepnote.com/@gleb-mikhaylov-15d6/blog-posts-nXDqO5CTQYOreNE53H3y-A)